In [ ]:
from math import exp, inf, log
import os
import random

import matplotlib.pyplot as mpl
import numpy as np
import pandas as pd
from scipy.misc import imread


In [ ]:
def fetch_data(train=True):
    X = []
    y = []
    for f in os.listdir('train/face' if train else 'test/face'):
        X.append(imread(f).ravel())
        y.append(1)

    for f in os.listdir('train/non-face' if train else 'test/non-face'):
        X.append(imread(f).ravel())
        y.append(0)
    
    return pd.DataFrame(X, columns=range(len(X[0]))), pd.Series(y)


def fetch_train():
    return fetch_data(True)    


def fetch_test():
    return fetch_data(False)


In [ ]:
def accuracy(h, y):
    # Accuracy = TP + TN / TP +TN +FP +FN
    tp = list(map(all, zip(h, y))).count(True)
    tf = list(map(any, zip(h, y))).count(False)
    return (tp + tf) / h.shape[0]


def precision(h, y):
    # Precision = TP / (TP + FP)
    tp = list(map(all, zip(h, y))).count(True)
    fp = list(zip(h, y)).count((1, 0))
    if tp + fp == 0:
        return 0.0
    return tp / (tp + fp)


def recall(h, y):
    # Recall = TP /(TP + FN)
    tp = list(map(all, zip(h, y))).count(True)
    fn = list(zip(h, y)).count((0, 1))
    if tp + fn == 0:
        return 0.0
    return tp / (tp + fn)


def f1(h, y):
    # F1 = 2 * (p * r / (p + r))
    p = precision(h, y)
    r = recall(h, y)
    if p+r == 0.0:
        return 0.0
    return 2 * (p * r / (p + r))


In [ ]:
class NaiveBayes:
    def __init__(self, X, y):
        self.X = X
        self.y = y

        self.yprior = {}
        self.y_indices = {}
        self.xcounts = {}

        self.class_counts = yvals = y.value_counts()
        for z in yvals:
            self.yprior[z] = yvals[z] / yvals.sum()
            self.y_indices[z] = [z == v for v in self.y]

        for z in yvals:
            # count x values by
            self.xcounts[z] = [self.X[c].count_values().to_dict() for c in self.X]

    def px(self, X, y):
        logpx = 0
        for x in X:
            logpx += log((self.xcounts[y].get(x, 0) + 1) / (self.class_counts[y] + 256))
        
        return logpx

    def py(self, y):
        return self.yprior[y]

    def classify(self, X):
        maxp = -inf
        cls = None
        for y in self.yprior.keys():
            p = self.py(y) + self.px(X, y)
            if p > maxp:
                maxp = p
                cls = y

        return cls

    def classify_all(self, X):
        return [self.classify(r) for _, r in X.iterrows()]


In [ ]:
def sigmoid(z):
    # print(z)
    try:
        return 1 / (1 + exp(-z))
    except OverflowError:
        return 0.0


def hw(X, w):
    return np.array(list(map(sigmoid, np.matmul(X, w))))


def cost(h, y):
    # cost (hw(x), y) = − [y log(hw (x)) + (1 − y) log(1 − hw (x))]
    # return - (y * log(h) + (1-y) * log(1-h))
    if y == 0:
        if h == 1.0:
            h -= 1e-6
        return -log(1-h)
    else:
        if h == 0.0:
            h += 1e-6
        return -log(h)


def costs(hyp, y):
    return np.array([cost(h, target) for (h, target) in zip(hyp, y)])


def batch_gradient_descent(X, y, nEpoch, alpha, lmbda=None):
    w = np.ones((X.shape[1],), dtype=np.float128)
    loss = -1
    for _ in np.arange(0, nEpoch):
        hypothesis = hw(X, w)
        loss = sum(costs(hypothesis, y))  # + sum(1/X.shape[0] * (w ** 2))

        error = hypothesis - y
        gradient = X.T.dot(error)

        if lmbda is None:
            w -= alpha * gradient
        else:
            w[0] -= alpha * gradient[0]
            w[1:] -= alpha * (gradient[1:] - lmbda * w[1:])

    print("epoch #{} alpha {} lambda {}, loss={:.7f}".format(nEpoch, alpha, lmbda, loss))

    return w


def stochastic_gradient_descent(X, y, nEpoch, alpha, lmbda=None):
    w = np.ones((X.shape[1],), dtype=np.float128)
    loss = -1
    for _ in np.arange(0, nEpoch):
        idx = random.randrange(X.shape[0])
        hypothesis = sigmoid(np.matmul(X[idx], w))
        loss = cost(hypothesis, y[idx])

        error = hypothesis - y
        gradient = X[idx].T.dot(error)

        if lmbda is None:
            w -= alpha * gradient
        else:
            w[0] -= alpha * gradient[0]
            w[1:] -= alpha * (gradient[1:] - lmbda * w[1:])

    print("epoch #{} alpha {} lambda {}, loss={:.7f}".format(nEpoch, alpha, lmbda, loss))

    return w

In [ ]:
trainX, trainY = fetch_train()
testX, testY = fetch_test()

In [ ]:
# Run Naive Bayes
naive_bayes = NaiveBayes(trainX, trainY)
nb_results = naive_bayes.classify(testX)

nb_accuracy = accuracy(nb_results, testY)
nb_precision = precision(nb_results, testY)
nb_recall = recall(nb_results, testY)
nb_f1 = f1(nb_results, testY)
nb_correct = [nb_results[i] == testY[i] for i in range(testX.shape[0])]

In [ ]:
# Run Batch Gradient Descent
bgd_results = batch_gradient_descent(trainX, trainY, 100, 0.1, 1)

bgd_accuracy = accuracy(bgd_results, testY)
bgd_precision = precision(bgd_results, testY)
bgd_recall = recall(bgd_results, testY)
bgd_f1 = f1(bgd_results, testY)

In [ ]:
# Run Stochastic Gradient Descent
sgd_results = stochastic_gradient_descent(trainX, trainY['y'], 1000, 0.1, 1)

sgd_accuracy = accuracy(sgd_results, testY['y'])
sgd_precision = precision(sgd_results, testY['y'])
sgd_recall = recall(sgd_results, testY['y'])
sgd_f1 = f1(sgd_results, testY['y'])